<a href="https://colab.research.google.com/github/KarthikMarella4/Maersk-Case-Study/blob/main/Maersk_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpy

In [3]:
import simpy
import random

In [4]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [5]:
SIM_TIME = 24 * 60

In [6]:
NUM_BERTHS = 2
NUM_CRANES = 2
NUM_TRUCKS = 3

In [7]:
class ContainerTerminal:
    def __init__(self, env, num_berths, num_cranes, num_trucks):
        self.env = env
        self.berths = simpy.Resource(env, num_berths)
        self.cranes = simpy.Resource(env, num_cranes)
        self.trucks = simpy.Resource(env, num_trucks)

    def berth_vessel(self, vessel):
        with self.berths.request() as request:
            yield request
            print(f"{self.env.now}: Vessel {vessel.name} berths")
            yield self.env.process(vessel.unload())

In [8]:
class Vessel:
    def __init__(self, env, name, terminal):
        self.env = env
        self.name = name
        self.terminal = terminal
        self.containers = 150

    def unload(self):
        with self.terminal.cranes.request() as request:
            yield request
            print(f"{self.env.now}: Quay crane starts unloading vessel {self.name}")
            while self.containers > 0:
                yield self.env.timeout(3)  # Time to move one container
                self.containers -= 1
                print(f"{self.env.now}: Container unloaded from vessel {self.name}, remaining: {self.containers}")

                with self.terminal.trucks.request() as truck_request:
                    yield truck_request
                    print(f"{self.env.now}: Truck starts transporting container from vessel {self.name}")
                    yield self.env.timeout(6)  # Time to transport and return
                    print(f"{self.env.now}: Truck returns to terminal from vessel {self.name}")

        print(f"{self.env.now}: Vessel {self.name} has finished unloading and leaves the berth")

In [9]:
def vessel_arrival(env, terminal):
    vessel_id = 0
    while True:
        yield env.timeout(random.expovariate(1 / (5 * 60)))  # Time between arrivals (5 hours)
        vessel = Vessel(env, f"Vessel_{vessel_id}", terminal)
        env.process(terminal.berth_vessel(vessel))
        vessel_id += 1

In [10]:
def run_simulation(simulation_time):
    env = simpy.Environment()
    terminal = ContainerTerminal(env, NUM_BERTHS, NUM_CRANES, NUM_TRUCKS)
    env.process(vessel_arrival(env, terminal))
    env.run(until=simulation_time)

In [11]:
if __name__ == "__main__":
    run_simulation(SIM_TIME)

306.0180861824403: Vessel Vessel_0 berths
306.0180861824403: Quay crane starts unloading vessel Vessel_0
309.0180861824403: Container unloaded from vessel Vessel_0, remaining: 149
309.0180861824403: Truck starts transporting container from vessel Vessel_0
313.61673789526196: Vessel Vessel_1 berths
313.61673789526196: Quay crane starts unloading vessel Vessel_1
315.0180861824403: Truck returns to terminal from vessel Vessel_0
316.61673789526196: Container unloaded from vessel Vessel_1, remaining: 149
316.61673789526196: Truck starts transporting container from vessel Vessel_1
318.0180861824403: Container unloaded from vessel Vessel_0, remaining: 148
318.0180861824403: Truck starts transporting container from vessel Vessel_0
322.61673789526196: Truck returns to terminal from vessel Vessel_1
324.0180861824403: Truck returns to terminal from vessel Vessel_0
325.61673789526196: Container unloaded from vessel Vessel_1, remaining: 148
325.61673789526196: Truck starts transporting container fr